In [10]:
import tkinter as tk
from tkinter import messagebox
import cv2
import os
from PIL import Image
import numpy as np
import mysql.connector

In [11]:
mydb=mysql.connector.connect(
host="localhost",
user="root",
password=""
)

print(mydb)

In [35]:
mydb=mysql.connector.connect(
host="localhost",
user="root",
password="",
database="students"
)
mycursor=mydb.cursor()
mycursor.execute("create table stu_datas(id int auto_increment primary key,name varchar(50),Department varchar(50),index_num int) ")


In [34]:
mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="students"
)

# Create a cursor object
mycursor = mydb.cursor()

# Define the SQL command to drop the table
table_name = "stu_datas"  # Replace with your table name
sql = f"DROP TABLE IF EXISTS {table_name}"

# Execute the SQL command
mycursor.execute(sql)

# Commit the transaction (optional)
mydb.commit()

# Close the cursor and connection
mycursor.close()
mydb.close()

In [36]:
window=tk.Tk()
window.title("Face recognization system")

l1 = tk.Label(window, text="Name", font=("Algerian", 25))
l1.grid(column=0,row=0)
t1=tk.Entry(window,width=50,bd=5)
t1.grid(column=1,row=0)   

l2 = tk.Label(window, text="Department", font=("Algerian", 25))
l2.grid(column=0,row=1)
t2=tk.Entry(window,width=50,bd=5)
t2.grid(column=1,row=1)   

l3 = tk.Label(window, text="Indexnumber", font=("Algerian", 25))
l3.grid(column=0,row=2)
t3=tk.Entry(window,width=50,bd=5)
t3.grid(column=1,row=2)   
def train_classifer():
    data_dir=("data")
    path=[os.path.join(data_dir,file) for file in os.listdir(data_dir)]
    
    faces=[]
    ids=[]
    
    for image in path:
        img=Image.open(image).convert('L')
        imageNp=np.array(img,'uint8')
        id=int(os.path.split(image)[1].split(".")[1])
        
        faces.append(imageNp)
        ids.append(id)
    ids=np.array(ids)
    #train and save
    clf=cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces,ids)
    clf.write("classifer.xml")
    cv2.destroyAllWindows()
    messagebox.showinfo('Result','collecting completed...')
    

        
b1=tk.Button(window,text="Training",font=("Algerian", 25),bg="Blue",fg="white",command=train_classifer)
b1.grid(column=0,row=3)


def generate_dataset():
    if(t1.get()=='' or t2.get()=='' or t3.get()=='' ):
        messagebox.showinfo('Result ','please provide complete of user')
    else:
        mydb = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="students"
        )
        mycursor = mydb.cursor()
        sql = "INSERT INTO stu_datas (name, Department, index_num) VALUES (%s, %s, %s)"
        val = (t1.get(), t2.get(), t3.get())
        mycursor.execute(sql, val)
        mydb.commit()

        mycursor.execute("SELECT id FROM stu_datas WHERE name=%s AND Department=%s AND index_num=%s", (t1.get(), t2.get(), t3.get()))
        myresult = mycursor.fetchone()
        id = myresult[0]
        
        face_clasifier=cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        def face_cropped(img):
            image=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            faces = face_clasifier.detectMultiScale(image, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))
            #scaling factir=1.3
            #minimum_negibour=5

            if faces is():
                return None
            for(x,y,h,w) in faces:
                cropped_face=img[y:y+h,x:x+w]
            return cropped_face

        cap=cv2.VideoCapture(0)
        
        img_id=0

        while True:
            ret,frame=cap.read()
            if face_cropped(frame) is not None:
                img_id+=1
                face=cv2.resize(face_cropped(frame),(200,200))
                face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
                file_path="data/user."+str(id)+"."+str(img_id)+".jpg"  
                cv2.imwrite(file_path,face)
                cv2.putText(face,str(img_id),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)

                cv2.imshow("Cropped image",face)
                if cv2.waitKey(1)==13 or int(img_id)==100:
                    break;
        cap.release()
        cv2.destroyAllWindows()
        messagebox.showinfo('Result','collecting completed click training button....')
            

        
        
        
    
b2=tk.Button(window,text="GetData",font=("Algerian", 25),bg="Blue",fg="white",command=generate_dataset)
b2.grid(column=1,row=3)


window.geometry("800x600")
window.mainloop()

In [37]:
window=tk.Tk()
window.title("Face recognization system")

def detect_face():
        def draw_boundry(img,classifier,scaleFactor,minNehighbors,color,text,cls):
            gray_image=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            feactures=classifier.detectMultiScale(gray_image, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

            coords=[]

            for(x,y,w,h) in feactures:
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),5)
                id,pred=cls.predict(gray_image[y:y+h,x:x+w])

                confidence=int(100*(1-pred/300))
                
                mydb=mysql.connector.connect(
                host="localhost",
                user="root",
                password="",
                database="students")
                mycursor=mydb.cursor()
                mycursor.execute("SELECT name from stu_datas where id="+str(id))
                s=mycursor.fetchone()
                s = ''.join(s)

                if confidence>77:
                    
                    cv2.putText(img,s,(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,color,1,cv2.LINE_AA)
                    
                else:
                    cv2.putText(img,"Unkown",(x,y-5),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,255),1,cv2.LINE_AA)

                coords.append((x, y, w, h))
            return coords

        def reconize(img,clf,faceCascade):
            coords=draw_boundry(img,faceCascade,1.1,10,(255,255,255),"Face",clf)
            return img

        faceCascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        clf=cv2.face.LBPHFaceRecognizer_create()
        clf.read("classifer.xml")

        video_cap=cv2.VideoCapture(0)

        while True:
            ret,img=video_cap.read()
            img= reconize(img,clf,faceCascade)
            cv2.imshow("face detection",img)

            if cv2.waitKey(1)==13:
                break

        video_cap.release()
        cv2.destroyAllWindows()
b1=tk.Button(window,text="Excisiting Student",font=("Algerian", 25),bg="Blue",fg="white",command=detect_face)
b1.grid(column=0,row=1)



window.geometry("800x600")
window.mainloop()

In [ ]:
import tkinter as tk
from tkinter import messagebox
import cv2
from PIL import Image
import numpy as np
import os
import mysql.connector
import time
import openpyxl
from openpyxl import load_workbook
from datetime import datetime

# Function to update attendance in Excel sheet
def update_attendance(name):
    print(f"Updating attendance for {name}")
    filename = 'Attendance_sheet.xlsx'
    date = datetime.now().strftime('%Y-%m-%d')
    Attendance = 'Present'

    if not os.path.exists(filename):
        print(f"{filename} does not exist. Creating a new workbook.")
        wb = openpyxl.Workbook()
        sheet = wb.active
        sheet.title = 'Attendance'
        sheet.append(['Name', 'Date', 'Status'])
        wb.save(filename)

    wb = load_workbook(filename)
    if 'Attendance' not in wb.sheetnames:
        print("Sheet 'Attendance' does not exist. Creating a new sheet.")
        sheet = wb.create_sheet('Attendance')
        sheet.append(['Name', 'Date', 'Status'])
    else:
        sheet = wb['Attendance']

    print(f"Appending data: {name}, {date}, {Attendance}")
    sheet.append([name, date, Attendance])
    
    try:
        wb.save(filename)
        print(f"Workbook saved successfully to {filename}")
    except PermissionError:
        alt_filename = f"Attendance_sheet_{int(time.time())}.xlsx"
        wb.save(alt_filename)
        messagebox.showinfo('File Error', f'Original file is open or cannot be accessed. Saved to {alt_filename}.')
        print(f"Workbook saved to alternate file {alt_filename}")

def new_student():
    new_window = tk.Toplevel(window)
    new_window.title("New Student Registration")
    
    l1 = tk.Label(new_window, text="Name", font=("Algerian", 25))
    l1.grid(column=0, row=0)
    t1 = tk.Entry(new_window, width=50, bd=5)
    t1.grid(column=1, row=0)   

    l2 = tk.Label(new_window, text="Department", font=("Algerian", 25))
    l2.grid(column=0, row=1)
    t2 = tk.Entry(new_window, width=50, bd=5)
    t2.grid(column=1, row=1)   

    l3 = tk.Label(new_window, text="Index Number", font=("Algerian", 25))
    l3.grid(column=0, row=2)
    t3 = tk.Entry(new_window, width=50, bd=5)
    t3.grid(column=1, row=2)   
    
    def train_classifer():
        data_dir = "data"
        path = [os.path.join(data_dir, file) for file in os.listdir(data_dir)]

        faces = []
        ids = []

        for image in path:
            img = Image.open(image).convert('L')
            imageNp = np.array(img, 'uint8')
            id = int(os.path.split(image)[1].split(".")[1])

            faces.append(imageNp)
            ids.append(id)
        ids = np.array(ids)
        
        clf = cv2.face.LBPHFaceRecognizer_create()
        clf.train(faces, ids)
        clf.write("classifer.xml")
        cv2.destroyAllWindows()
        messagebox.showinfo('Result', 'Training completed...')
        new_window.destroy()  # Close the new student window after training is completed
    
    b1 = tk.Button(new_window, text="Training", font=("Algerian", 25), bg="Blue", fg="white", command=train_classifer)
    b1.grid(column=0, row=3)
    
    def generate_dataset():
        if t1.get() == '' or t2.get() == '' or t3.get() == '':
            messagebox.showinfo('Result', 'Please provide complete information')
        else:
            mydb = mysql.connector.connect(
                host="localhost",
                user="root",
                password="",
                database="students_details"
            )
            mycursor = mydb.cursor()
            sql = "INSERT INTO stu_datas (name, department, index_num) VALUES (%s, %s, %s)"
            val = (t1.get(), t2.get(), t3.get())
            mycursor.execute(sql, val)
            mydb.commit()

            mycursor.execute("SELECT id FROM stu_datas WHERE name=%s AND department=%s AND index_num=%s", (t1.get(), t2.get(), t3.get()))
            myresult = mycursor.fetchone()
            id = myresult[0]

            face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            
            def face_cropped(img):
                gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                faces = face_classifier.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

                if len(faces) == 0:
                    return None
                for (x, y, w, h) in faces:
                    cropped_face = img[y:y+h, x:x+w]
                return cropped_face

            cap = cv2.VideoCapture(0)
            img_id = 0

            while True:
                ret, frame = cap.read()
                if face_cropped(frame) is not None:
                    img_id += 1
                    face = cv2.resize(face_cropped(frame), (200, 200))
                    face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                    file_path = "data/user." + str(id) + "." + str(img_id) + ".jpg"  
                    cv2.imwrite(file_path, face)
                    cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

                    cv2.imshow("Cropped image", face)
                    if cv2.waitKey(1) == 13 or int(img_id) == 100:
                        break
            cap.release()
            cv2.destroyAllWindows()
            messagebox.showinfo('Result', 'Data collection completed. Click the training button.')

    b2 = tk.Button(new_window, text="Get Data", font=("Algerian", 25), bg="Blue", fg="white", command=generate_dataset)
    b2.grid(column=1, row=3)

def detect_face():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        features = classifier.detectMultiScale(gray, scaleFactor, minNeighbors, minSize=(30, 30))

        name = "Unknown"
        for (x, y, w, h) in features:
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
            id, pred = clf.predict(gray[y:y+h, x:x+w])
            confidence = int(100 * (1 - pred / 300))

            mydb = mysql.connector.connect(
                host="localhost",
                user="root",
                password="",
                database="students_details"
            )
            mycursor = mydb.cursor()
            mycursor.execute("SELECT name FROM stu_datas WHERE id=" + str(id))
            s = mycursor.fetchone()
            if s is not None:
                s = ''.join(s) 

            if confidence > 77:
                cv2.putText(img, s, (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
                name = s
            else:
                cv2.putText(img, "Unknown", (x, y-5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1, cv2.LINE_AA)

        return name
        
    def recognize(img, clf, faceCascade):
        name = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
        return img, name

    faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifer.xml")

    video_cap = cv2.VideoCapture(0)
    cv2.namedWindow("Face Detection")

    marked = False
    start_time = time.time()
    while True:
        ret, img = video_cap.read()
        img, name = recognize(img, clf, faceCascade)
        cv2.imshow("Face Detection", img)

        if cv2.waitKey(1) == 13:  # Press Enter key to exit
            break

        if name != "Unknown":
            update_attendance(name)  # Update the attendance in the Excel sheet
            messagebox.showinfo('Attendance Marked', f'{name} marked attendance!')
            marked = True
            break

        if time.time() - start_time > 300:  # 5 minutes
            messagebox.showinfo('Result', 'No face detected for 5 minutes. Please enter your details by clicking the New Student button.')
            break

    video_cap.release()
    cv2.destroyAllWindows()
    
    return marked

def mark_attendance():
    if detect_face():
        print("Attendance marked successfully.")
    else:
        print("No attendance was marked.")

def main():
    global window
    window = tk.Tk()
    window.title("Face Recognition System")

    l1 = tk.Label(window, text="Select an option", font=("Algerian", 25))
    l1.grid(column=1, row=0)

    b1 = tk.Button(window, text="Existing Student", font=("Algerian", 25), bg="Blue", fg="white", command=mark_attendance)
    b1.grid(column=1, row=1)

    b2 = tk.Button(window, text="New Student", font=("Algerian", 25), bg="Blue", fg="white", command=new_student)
    b2.grid(column=2, row=1)

    window.geometry("800x600")
    window.mainloop()

if __name__ == "__main__":
    main()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\envs\ip\lib\tkinter\__init__.py", line 1705, in __call__
    return self.func(*args)
  File "C:\Users\vimal\AppData\Local\Temp\ipykernel_12224\651573355.py", line 216, in mark_attendance
    if detect_face():
  File "C:\Users\vimal\AppData\Local\Temp\ipykernel_12224\651573355.py", line 194, in detect_face
    img, name = recognize(img, clf, faceCascade)
  File "C:\Users\vimal\AppData\Local\Temp\ipykernel_12224\651573355.py", line 180, in recognize
    name = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), "Face", clf)
  File "C:\Users\vimal\AppData\Local\Temp\ipykernel_12224\651573355.py", line 163, in draw_boundary
    database="students_details"
  File "C:\Users\vimal\AppData\Roaming\Python\Python37\site-packages\mysql\connector\__init__.py", line 179, in connect
    return MySQLConnection(*args, **kwargs)
  File "C:\Users\vimal\AppData\Roaming\Python\Python37\site-packages\mys